In [13]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from threading import Lock, Timer
import json,time
import pandas as pd
from unidecode import unidecode
import pymysql
from DBUtils.PooledDB import PooledDB

In [14]:
ckey="QG4kEAhNnqQAZ5DMnCxOa9MJk"
csecret="8DlPtLvMHehu3ULe7k6oUQGY9ngk7Be0UNsOUOvcsb3a4mUFPd"
atoken="764487385069608960-vKAW7gm7hkb9hxSsT0Ne9xybYhkHQQT"
asecret="xDYG0mIfDtzDRCabMVS8davPlMVOkJIFoDdlqz40XHGXi"
analyzer = SentimentIntensityAnalyzer()
MysqlHost = "192.168.6.50"
MysqlPassword = '123456'
MysqlDatabase = "py_orderlist"

In [15]:
POOL = PooledDB(
    creator=pymysql,  # 使用链接数据库的模块
    maxconnections=100,  # 连接池允许的最大连接数，0和None表示不限制连接数
    mincached=2,  # 初始化时，链接池中至少创建的空闲的链接，0表示不创建
    maxcached=5,  # 链接池中最多闲置的链接，0和None不限制
    maxshared=3,
    # 链接池中最多共享的链接数量，0和None表示全部共享。PS: 无用，因为pymysql和MySQLdb等模块的 threadsafety都为1，所有值无论设置为多少，_maxcached永远为0，所以永远是所有链接都共享。
    blocking=True,  # 连接池中如果没有可用连接后，是否阻塞等待。True，等待；False，不等待然后报错
    maxusage=None,  # 一个链接最多被重复使用的次数，None表示无限制
    setsession=[],  # 开始会话前执行的命令列表。
    ping=0,  # ping MySQL服务端，检查是否服务可用。
    host=MysqlHost,
    port=3306,
    # user='quantify',   # 量化app正式
    # password="58h59PIef2Jj7ozJ",
    user='admin',
    password=MysqlPassword,
    database="py_orderlist",
    charset='utf8'
)

In [16]:
conn = POOL.connection()
cur = conn.cursor()

In [17]:
lock = Lock()

In [18]:
class listener(StreamListener):

    data = []
    lock = None
    def __init__(self, lock):

        # create lock
        self.lock = lock
        # init timer for database save
        self.save_in_database()

        # call __inint__ of super class
        super().__init__()

    def save_in_database(self):

        # set a timer (1 second)
        Timer(1, self.save_in_database).start()

        # with lock, if there's data, save in transaction using one bulk query
        with self.lock:
            if len(self.data):
                print(self.data)
                insertsql = "INSERT INTO twitter_stream(time_vs,tweet,sentiment) VALUES(%s, %s, %s) "
                cur.executemany(insertsql,self.data)
                conn.commit()
                self.data =[]

    def on_data(self, data):
        try:
            #print('data')
            data = json.loads(data)
            # there are records like that:
            # {'limit': {'track': 14667, 'timestamp_ms': '1520216832822'}}
            if 'truncated' not in data:
                #print(data)
                return True
            if data['truncated']:
                tweet = unidecode(data['extended_tweet']['full_text'])
            else:
                tweet = unidecode(data['text'])
            time_ms=time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(data['timestamp_ms'])/1000))
            vs = analyzer.polarity_scores(tweet)
            sentiment = vs['compound']
            #print(time_ms,tweet,sentiment)

            # append to data list (to be saved every 1 second)
            with self.lock:
                self.data.append((time_ms, tweet, sentiment))

        except KeyError as e:
            #print(data)
            print(str(e))
        return True

    def on_error(self, status):
        print(status)

In [19]:
auth = OAuthHandler(ckey, csecret)

In [20]:
auth.set_access_token(atoken, asecret)

In [21]:
twitterStream = Stream(auth, listener(lock))

In [10]:
twitterStream.filter(track=["ethereum"])

[('2021-09-13 09:05:43', "This clone is not officially affiliated with @isle_of_NaN's project. Just a fun derivative :)\n\nPrice: .07 ETH\n\nLink @opensea:\nhttps://t.co/oJQR0D40X5\n\n#openseanft #digitalart #ethereum #cryptoart #cryptocurrency #boredbananas #boredbananas nft #bitcoin #openseaio #nft #nftart", 0.743), ('2021-09-13 09:05:44', 'RT @MkeyBsc: Mkey Big Airdrop \n    \n  $60 for Doing Tasks ( 60 $Mkey )\n  $10 for each User Join by Your Referral Link ( 10 $Mkey ) \n\n-Fr...', 0.296)]


KeyboardInterrupt: 

In [ ]:
# new=pd.DataFrame({'time_ms':1,'tweet':2,'sentiment':3},index=[1])
# df=df.append(new,ignore_index=True)

In [11]:
df = pd.read_sql("SELECT * FROM twitter_stream where time_vs>'2021-09-10 08:00:00' ORDER BY id DESC", conn)

In [29]:
df

,id,tweet,sentiment
time_vs,,,
2021-09-10 09:07:06,10335,@AdlerSofie @devikins_game @moonlabsstudios @k...,0.0000
2021-09-10 09:07:06,19206,@AdlerSofie @devikins_game @moonlabsstudios @k...,0.0000
2021-09-10 09:07:06,17901,@AdlerSofie @devikins_game @moonlabsstudios @k...,0.0000
2021-09-10 09:07:06,16839,@AdlerSofie @devikins_game @moonlabsstudios @k...,0.0000
2021-09-10 09:07:06,14402,@AdlerSofie @devikins_game @moonlabsstudios @k...,0.0000
...,...,...,...
2021-09-13 09:00:49,392777,RT @Cryptolyst_: #Giveaway \n\nI have some mo...,0.8516
2021-09-13 09:00:49,392776,RT @DASL_Me: R3 Corda Now Has a Bridge to Publ...,0.0000
2021-09-13 09:00:49,392778,@Thantzin3690 @meethiri272,0.0000


In [31]:
df.to_csv("舆情数据.csv")

In [ ]:
df.drop_duplicates(subset=['tweet'],keep='first',inplace=True)

In [ ]:
len(df)

In [ ]:
df['sentiment_smooth']=df['sentiment'].rolling(2000).mean()

df[['sentiment_smooth']].plot(figsize=(20,8))

In [ ]:
now_time=time.localtime(time.time())

In [ ]:
import datetime
def clear_cache(n):
     #获取n天前时间
    today=datetime.datetime.now()
    offset=datetime.timedelta(days=n)
    before_date=(today-offset).strftime("%Y-%m-%d %H:%M:%S")
    delete_sql="delete  from tweet_stream where time_vs<%s"
    cur.execute(delete_sql,(before_date))
    conn.commit()